# EndSARS Twitter Analysis

Sentiment and Network Analysis of the #EndSARS protest movement that occured in 2020 in Nigeria. 

## SetUp

### Import Python Libraries

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns#
import yaml
import os
import json
import re
import ast
import csv
import gensim
from searchtweets import ResultStream, gen_request_parameters, load_credentials, collect_results
# import fastText as ft
from sklearn.model_selection import train_test_split
import numpy as np
from datetime import datetime, timedelta


%matplotlib inline
pd.set_option("display.max_rows", 500)

C:\Users\USER\anaconda3\envs\endsarsproject\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


### Connecting to Twitter

I suggest you use Twitter's official searchtweets api which is easy to use and reliable.
<br>In order to get past the tweets retrieval limit, you might also need to  use the premium search api.

- Create a hidden yaml file (.yaml) which contains details about your twitter api keys
- Create a variable that stores the twitter api credentials from the hiddem yaml file above.

**Keywords**: EndSARS, Soro Soke, Police Brutality, lekki toll gate, anti-robbery, lekki massacre, End bad governance,
End swat, buhari is a bad boy, sanwa-olu is a murderer, #LekkiMassacre

Helpful links:
- https://developer.twitter.com/en/docs/twitter-api/tweets/search/api-reference/get-tweets-search-recent
- https://github.com/twitterdev/search-tweets-python/tree/v2#using-the-twitter-search-apis-python-wrapper

In [3]:
search_args = load_credentials("tweetkeys.yaml",
                                       yaml_key="search_tweets_v2",
                                       env_overwrite=False)


In [4]:
query = gen_request_parameters('#EndSars',results_per_call=100)
print(query)



{"query": "#EndSars", "max_results": 100}


In [5]:
tweets = collect_results(query,
                         max_tweets=100,
                         result_stream_args=search_args) 

In [6]:
tweets

[{'id': '1380787080364691456',
  'text': "RT @bengbenro: Hope it's clear to you now that #EndSARS was a franchise. What will you do now with all the curses you rained on those you m…"},
 {'id': '1380787027130601472',
  'text': 'RT @ifemosumichael: 17yr old Kemisola Ogunniye has been in detention since 24/10/20, she was arrested by @PoliceNG on her way to get medica…'},
 {'id': '1380787024882454528',
  'text': 'RT @obynonwachineke: In the civilised world, People mourn when a Soldier or a Police   man is killed. But in Nigeria 🇳🇬 the People Rejoices…'},
 {'id': '1380786969626693633',
  'text': '@Sammykings13 @renoomokri @MBuhari We are not part of your lazy generations.....\nWe work hard to earn what we eat I swear not part of you useless zobo colour flag called #ENDSARS agitators.....\nOponu https://t.co/z2MGZ6xbQP'},
 {'id': '1380786946017013760',
  'text': 'I love Aljazeera too much mooe so after they made up for their initial bogus report on #Endsars on 21st oct 2020. https://t.co/GS

In [7]:
[print(tweet['text'], end='\n\n') for tweet in tweets[0:10]]


RT @bengbenro: Hope it's clear to you now that #EndSARS was a franchise. What will you do now with all the curses you rained on those you m…

RT @ifemosumichael: 17yr old Kemisola Ogunniye has been in detention since 24/10/20, she was arrested by @PoliceNG on her way to get medica…

RT @obynonwachineke: In the civilised world, People mourn when a Soldier or a Police   man is killed. But in Nigeria 🇳🇬 the People Rejoices…

@Sammykings13 @renoomokri @MBuhari We are not part of your lazy generations.....
We work hard to earn what we eat I swear not part of you useless zobo colour flag called #ENDSARS agitators.....
Oponu https://t.co/z2MGZ6xbQP

I love Aljazeera too much mooe so after they made up for their initial bogus report on #Endsars on 21st oct 2020. https://t.co/GSiEd83axc

RT @botofaweirdo: Anti-people policies cancellation #EndSARS #EndPoliceBrutality

RT @Imoteda: Daily Reminder:

At this time on 20/10/20, the army shot live rounds at unarmed protesters at the Lekki Toll Gate. 

[None, None, None, None, None, None, None, None, None, None]

In [8]:
query = gen_request_parameters('#EndSars',results_per_call=100, tweet_fields = "id,created_at,text,geo,author_id,referenced_tweets,in_reply_to_user_id,entities,public_metrics,possibly_sensitive")
print(query)


{"query": "#EndSars", "max_results": 100, "tweet.fields": "id,created_at,text,geo,author_id,referenced_tweets,in_reply_to_user_id,entities,public_metrics,possibly_sensitive"}


In [9]:
tweets = collect_results(query,
                        max_tweets=500,
                        result_stream_args=search_args)
df1=[print(tweet['text'], end='\n\n') for tweet in tweets[0:10]]
df1


RT @NdidiUwechue: This is in line with United Nations guidelines. Sign this petition if you want to #End1999Constitution to give Freedom th…

RT @kingz_autos: Contact me now to secure yours. You procastinate today, you miss out tomorrow..  #Properties #realestateagent #RealEstate…

RT @Otoyosam: @Letter_to_Jack I wrote this yesterday about #EndSARS . In the quest to fight for justice or whatever social inequality, ther…

RT @bengbenro: Hope it's clear to you now that #EndSARS was a franchise. What will you do now with all the curses you rained on those you m…

RT @ifemosumichael: 17yr old Kemisola Ogunniye has been in detention since 24/10/20, she was arrested by @PoliceNG on her way to get medica…

RT @obynonwachineke: In the civilised world, People mourn when a Soldier or a Police   man is killed. But in Nigeria 🇳🇬 the People Rejoices…

@Sammykings13 @renoomokri @MBuhari We are not part of your lazy generations.....
We work hard to earn what we eat I swear not part of you useless zob

[None, None, None, None, None, None, None, None, None, None]

In [10]:
tweet_df1 = pd.DataFrame(tweets)
tweet_df1.head()


,author_id,possibly_sensitive,entities,public_metrics,created_at,text,referenced_tweets,id,in_reply_to_user_id,newest_id,oldest_id,result_count,next_token,geo
0,2982863907,False,"{'annotations': [{'start': 39, 'end': 52, 'pro...","{'retweet_count': 8, 'reply_count': 0, 'like_c...",2021-04-10T07:38:15.000Z,RT @NdidiUwechue: This is in line with United ...,"[{'type': 'retweeted', 'id': '1380635128410505...",1380787194026205184,NaN,NaN,NaN,NaN,NaN,NaN
1,4872881662,False,"{'hashtags': [{'start': 98, 'end': 109, 'tag':...","{'retweet_count': 2, 'reply_count': 0, 'like_c...",2021-04-10T07:38:02.000Z,RT @kingz_autos: Contact me now to secure your...,"[{'type': 'retweeted', 'id': '1377721077019897...",1380787139391152129,NaN,NaN,NaN,NaN,NaN,NaN
2,1325706077237882881,False,"{'hashtags': [{'start': 59, 'end': 67, 'tag': ...","{'retweet_count': 4, 'reply_count': 0, 'like_c...",2021-04-10T07:37:58.000Z,RT @Otoyosam: @Letter_to_Jack I wrote this yes...,"[{'type': 'retweeted', 'id': '1380783417936216...",1380787123452792835,NaN,NaN,NaN,NaN,NaN,NaN
3,288192640,False,"{'hashtags': [{'start': 47, 'end': 55, 'tag': ...","{'retweet_count': 120, 'reply_count': 0, 'like...",2021-04-10T07:37:48.000Z,RT @bengbenro: Hope it's clear to you now that...,"[{'type': 'retweeted', 'id': '1380575755235692...",1380787080364691456,NaN,NaN,NaN,NaN,NaN,NaN
4,1644626130,False,"{'annotations': [{'start': 29, 'end': 45, 'pro...","{'retweet_count': 5713, 'reply_count': 0, 'lik...",2021-04-10T07:37:35.000Z,RT @ifemosumichael: 17yr old Kemisola Ogunniye...,"[{'type': 'retweeted', 'id': '1378270507817123...",1380787027130601472,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
tweet_fields="id,author_id,created_at,text,geo,referenced_tweets,in_reply_to_user_id,entities,public_metrics.lang,possiby_sensitive" 
user_fields="id,created_at,location,username,verified,public_metrics"
place_fields="country,country_code,full_name,geo,id,name,place_type"

In [12]:
query2 = gen_request_parameters('#EndSars',results_per_call=100,expansions="author_id,entities.mentions.username,geo.place_id,in_reply_to_user_id,referenced_tweets.id,referenced_tweets.id.author_id")
print(query2)


{"query": "#EndSars", "max_results": 100, "expansions": "author_id,entities.mentions.username,geo.place_id,in_reply_to_user_id,referenced_tweets.id,referenced_tweets.id.author_id"}


In [13]:
tweets2 = collect_results(query2,
                        max_tweets=500,
                        result_stream_args=search_args)
[print(tweet['text'], end='\n\n') for tweet in tweets2[0:10]]



RT @mrs_savyy: Just few thoughts regarding allegations against Igbos and #MaziNnamdiKanu. I hope we can reach out to more people 💖 
Full vi…

RT @NdidiUwechue: This is in line with United Nations guidelines. Sign this petition if you want to #End1999Constitution to give Freedom th…

RT @kingz_autos: Contact me now to secure yours. You procastinate today, you miss out tomorrow..  #Properties #realestateagent #RealEstate…

RT @Otoyosam: @Letter_to_Jack I wrote this yesterday about #EndSARS . In the quest to fight for justice or whatever social inequality, ther…

RT @bengbenro: Hope it's clear to you now that #EndSARS was a franchise. What will you do now with all the curses you rained on those you m…

RT @ifemosumichael: 17yr old Kemisola Ogunniye has been in detention since 24/10/20, she was arrested by @PoliceNG on her way to get medica…

RT @obynonwachineke: In the civilised world, People mourn when a Soldier or a Police   man is killed. But in Nigeria 🇳🇬 the People Rejoices…

@Sammyk

[None, None, None, None, None, None, None, None, None, None]

In [14]:
tweet_df2= pd.DataFrame(tweets)
tweet_df2.head()



,author_id,possibly_sensitive,entities,public_metrics,created_at,text,referenced_tweets,id,in_reply_to_user_id,newest_id,oldest_id,result_count,next_token,geo
0,2982863907,False,"{'annotations': [{'start': 39, 'end': 52, 'pro...","{'retweet_count': 8, 'reply_count': 0, 'like_c...",2021-04-10T07:38:15.000Z,RT @NdidiUwechue: This is in line with United ...,"[{'type': 'retweeted', 'id': '1380635128410505...",1380787194026205184,NaN,NaN,NaN,NaN,NaN,NaN
1,4872881662,False,"{'hashtags': [{'start': 98, 'end': 109, 'tag':...","{'retweet_count': 2, 'reply_count': 0, 'like_c...",2021-04-10T07:38:02.000Z,RT @kingz_autos: Contact me now to secure your...,"[{'type': 'retweeted', 'id': '1377721077019897...",1380787139391152129,NaN,NaN,NaN,NaN,NaN,NaN
2,1325706077237882881,False,"{'hashtags': [{'start': 59, 'end': 67, 'tag': ...","{'retweet_count': 4, 'reply_count': 0, 'like_c...",2021-04-10T07:37:58.000Z,RT @Otoyosam: @Letter_to_Jack I wrote this yes...,"[{'type': 'retweeted', 'id': '1380783417936216...",1380787123452792835,NaN,NaN,NaN,NaN,NaN,NaN
3,288192640,False,"{'hashtags': [{'start': 47, 'end': 55, 'tag': ...","{'retweet_count': 120, 'reply_count': 0, 'like...",2021-04-10T07:37:48.000Z,RT @bengbenro: Hope it's clear to you now that...,"[{'type': 'retweeted', 'id': '1380575755235692...",1380787080364691456,NaN,NaN,NaN,NaN,NaN,NaN
4,1644626130,False,"{'annotations': [{'start': 29, 'end': 45, 'pro...","{'retweet_count': 5713, 'reply_count': 0, 'lik...",2021-04-10T07:37:35.000Z,RT @ifemosumichael: 17yr old Kemisola Ogunniye...,"[{'type': 'retweeted', 'id': '1378270507817123...",1380787027130601472,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
query3 = gen_request_parameters('#EndSars',results_per_call=100,user_fields="id,created_at,location,username,verified,public_metrics")
print(query3)
tweets3 = collect_results(query3,
                        max_tweets=500,
                        result_stream_args=search_args)
[print(tweet['text'], end='\n\n') for tweet in tweets3[0:10]]


{"query": "#EndSars", "max_results": 100, "user.fields": "id,created_at,location,username,verified,public_metrics"}
RT @mrs_savyy: Just few thoughts regarding allegations against Igbos and #MaziNnamdiKanu. I hope we can reach out to more people 💖 
Full vi…

RT @NdidiUwechue: This is in line with United Nations guidelines. Sign this petition if you want to #End1999Constitution to give Freedom th…

RT @kingz_autos: Contact me now to secure yours. You procastinate today, you miss out tomorrow..  #Properties #realestateagent #RealEstate…

RT @Otoyosam: @Letter_to_Jack I wrote this yesterday about #EndSARS . In the quest to fight for justice or whatever social inequality, ther…

RT @bengbenro: Hope it's clear to you now that #EndSARS was a franchise. What will you do now with all the curses you rained on those you m…

RT @ifemosumichael: 17yr old Kemisola Ogunniye has been in detention since 24/10/20, she was arrested by @PoliceNG on her way to get medica…

RT @obynonwachineke: In the civil

[None, None, None, None, None, None, None, None, None, None]

In [16]:
tweet_df3= pd.DataFrame(tweets3)
tweet_df3.head()



,id,text,newest_id,oldest_id,result_count,next_token
0,1380787269427167233,RT @mrs_savyy: Just few thoughts regarding all...,NaN,NaN,NaN,NaN
1,1380787194026205184,RT @NdidiUwechue: This is in line with United ...,NaN,NaN,NaN,NaN
2,1380787139391152129,RT @kingz_autos: Contact me now to secure your...,NaN,NaN,NaN,NaN
3,1380787123452792835,RT @Otoyosam: @Letter_to_Jack I wrote this yes...,NaN,NaN,NaN,NaN
4,1380787080364691456,RT @bengbenro: Hope it's clear to you now that...,NaN,NaN,NaN,NaN


In [17]:
query4 = gen_request_parameters('#EndSars',results_per_call=100,place_fields="country,country_code,full_name,geo,id,name,place_type")
print(query4)
tweets4 = collect_results(query4,
                        max_tweets=500,
                        result_stream_args=search_args)


[print(tweet['text'], end='\n\n') for tweet in tweets4[0:10]]


{"query": "#EndSars", "max_results": 100, "place.fields": "country,country_code,full_name,geo,id,name,place_type"}
@Otoyosam @endsarsbot_ @Letter_to_Jack I don't just know!
For all those who keep saying that we they used us to cash out!, We were just pawns in a game!
A movement that happens with all spontaneity, was a chess game!
I tire for us black man!
#EndSARS #Nigeria

RT @mrs_savyy: Just few thoughts regarding allegations against Igbos and #MaziNnamdiKanu. I hope we can reach out to more people 💖 
Full vi…

RT @NdidiUwechue: This is in line with United Nations guidelines. Sign this petition if you want to #End1999Constitution to give Freedom th…

RT @kingz_autos: Contact me now to secure yours. You procastinate today, you miss out tomorrow..  #Properties #realestateagent #RealEstate…

RT @Otoyosam: @Letter_to_Jack I wrote this yesterday about #EndSARS . In the quest to fight for justice or whatever social inequality, ther…

RT @bengbenro: Hope it's clear to you now that #EndSARS w

[None, None, None, None, None, None, None, None, None, None]

In [18]:
tweet_df4= pd.DataFrame(tweets4)
tweet_df4.head()

,id,text,newest_id,oldest_id,result_count,next_token
0,1380787384455946242,@Otoyosam @endsarsbot_ @Letter_to_Jack I don't...,NaN,NaN,NaN,NaN
1,1380787269427167233,RT @mrs_savyy: Just few thoughts regarding all...,NaN,NaN,NaN,NaN
2,1380787194026205184,RT @NdidiUwechue: This is in line with United ...,NaN,NaN,NaN,NaN
3,1380787139391152129,RT @kingz_autos: Contact me now to secure your...,NaN,NaN,NaN,NaN
4,1380787123452792835,RT @Otoyosam: @Letter_to_Jack I wrote this yes...,NaN,NaN,NaN,NaN


In [19]:
frames = [tweet_df1, tweet_df2,tweet_df3,tweet_df4]
result = pd.concat(frames)
result



,author_id,possibly_sensitive,entities,public_metrics,created_at,text,referenced_tweets,id,in_reply_to_user_id,newest_id,oldest_id,result_count,next_token,geo
0,2982863907,False,"{'annotations': [{'start': 39, 'end': 52, 'pro...","{'retweet_count': 8, 'reply_count': 0, 'like_c...",2021-04-10T07:38:15.000Z,RT @NdidiUwechue: This is in line with United ...,"[{'type': 'retweeted', 'id': '1380635128410505...",1380787194026205184,NaN,NaN,NaN,NaN,NaN,NaN
1,4872881662,False,"{'hashtags': [{'start': 98, 'end': 109, 'tag':...","{'retweet_count': 2, 'reply_count': 0, 'like_c...",2021-04-10T07:38:02.000Z,RT @kingz_autos: Contact me now to secure your...,"[{'type': 'retweeted', 'id': '1377721077019897...",1380787139391152129,NaN,NaN,NaN,NaN,NaN,NaN
2,1325706077237882881,False,"{'hashtags': [{'start': 59, 'end': 67, 'tag': ...","{'retweet_count': 4, 'reply_count': 0, 'like_c...",2021-04-10T07:37:58.000Z,RT @Otoyosam: @Letter_to_Jack I wrote this yes...,"[{'type': 'retweeted', 'id': '1380783417936216...",1380787123452792835,NaN,NaN,NaN,NaN,NaN,NaN
3,288192640,False,"{'hashtags': [{'start': 47, 'end': 55, 'tag': ...","{'retweet_count': 120, 'reply_count': 0, 'like...",2021-04-10T07:37:48.000Z,RT @bengbenro: Hope it's clear to you now that...,"[{'type': 'retweeted', 'id': '1380575755235692...",1380787080364691456,NaN,NaN,NaN,NaN,NaN,NaN
4,1644626130,False,"{'annotations': [{'start': 29, 'end': 45, 'pro...","{'retweet_count': 5713, 'reply_count': 0, 'lik...",2021-04-10T07:37:35.000Z,RT @ifemosumichael: 17yr old Kemisola Ogunniye...,"[{'type': 'retweeted', 'id': '1378270507817123...",1380787027130601472,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,NaN,NaN,NaN,NaN,NaN,"RT @obynonwachineke: In the civilised world, P...",NaN,1380734607042220033,NaN,NaN,NaN,NaN,NaN,NaN
501,NaN,NaN,NaN,NaN,NaN,RT @Unklescarr: This bolt driver tried to kidn...,NaN,1380734502176186369,NaN,NaN,NaN,NaN,NaN,NaN
502,NaN,NaN,NaN,NaN,NaN,@channelstv This irresponsible APC govt is not...,NaN,1380734300279164929,NaN,NaN,NaN,NaN,NaN,NaN
503,NaN,NaN,NaN,NaN,NaN,@OluwatobaAmo #endsars #5for5 #EndPoliceBrutal...,NaN,1380734012843388934,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
search_args = load_credentials("tweetkeys1.yaml",
                                       yaml_key="search_tweets_v2",
                                       env_overwrite=False)


In [27]:
querry = gen_request_parameters("(#EndSARS OR #SoroSoke OR #EndPoliceBrutality OR #lekkitollgate OR anti-robbery OR #lekki massacre OR #Endbadgovernance OR #Endswat OR buhari is a bad boy OR sanwa-olu is a murderer OR #LekkiMassacre) lang:en -is:retweet",results_per_call=500, 
                                start_time ="2020-01-01", end_time="2020-01-08"
                                ,
                                tweet_fields="id,author_id,created_at,text,geo,referenced_tweets,in_reply_to_user_id,entities,public_metrics,lang,possibly_sensitive",
                                expansions="author_id,entities.mentions.username,geo.place_id,in_reply_to_user_id,referenced_tweets.id,referenced_tweets.id.author_id",
                                user_fields="id,created_at,location,username,verified,public_metrics",
                                place_fields="country,country_code,full_name,geo,id,name,place_type")


tweetss = collect_results(querry,
                        max_tweets=6000,
                        result_stream_args=search_args)


[print(tweet['text'], end='\n\n') for tweet in tweetss[0:10]]
tweet_df5= pd.DataFrame(tweetss)
tweet_df5.head()

With Faces Backing The Camera Unlike How Civilians Are Paraded. #EndSARS #EndImpunity https://t.co/lMGEGYUfrv

@SubDeliveryZone Special Anti Robbery Squad
They are hungry and raw criminals

Personnel of this office conducted house visitation and initiated dialogue re Anti-Robbery Safety Tips and other plans and programs of this office at Brgy. D' Alarcio, Laoac, Pangasinan

#PNPKakampiMo... https://t.co/sgDnUG2fSJ

As part of my new year contribution to bettering this country. I think @PoliceNG should introduce bodycams and patrol car cams to its security operatives. This will curtail a lot and improve the overall security of Nigerians. Justice for All
@segalink 
@PoliceNG_CRU
#EndSARS

This is not acceptable.
We want their faces and names boldly spelt out.
If it were to be civilian suspects, you will cheerfully beam the floodlights on their faces with lots of cameras and pressmen.
If you are protecting rogue officers, you are not ready to end impunity.
#EndSARS https://t.co/235zDTPrxN

,possibly_sensitive,public_metrics,created_at,entities,id,text,lang,geo,author_id,referenced_tweets,in_reply_to_user_id,users,places,tweets,newest_id,oldest_id,result_count
0,False,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",2020-01-07T17:50:12.000Z,"{'urls': [{'start': 86, 'end': 109, 'url': 'ht...",1214605164826562560,With Faces Backing The Camera Unlike How Civil...,en,{'place_id': '014326ae7dbb50ee'},326132860,"[{'type': 'quoted', 'id': '1214488973038497792'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,False,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2020-01-07T16:26:55.000Z,"{'mentions': [{'start': 0, 'end': 16, 'usernam...",1214584205201547265,@SubDeliveryZone Special Anti Robbery Squad\nT...,en,NaN,1053931388037201920,"[{'type': 'replied_to', 'id': '121458356651809...",892400664638304256,NaN,NaN,NaN,NaN,NaN,NaN
2,False,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2020-01-07T15:52:36.000Z,"{'urls': [{'start': 204, 'end': 227, 'url': 'h...",1214575569603424257,Personnel of this office conducted house visit...,en,NaN,2387886752,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,False,"{'retweet_count': 2, 'reply_count': 4, 'like_c...",2020-01-07T15:50:33.000Z,"{'hashtags': [{'start': 267, 'end': 275, 'tag'...",1214575055260262401,As part of my new year contribution to betteri...,en,NaN,290453376,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,False,"{'retweet_count': 5, 'reply_count': 1, 'like_c...",2020-01-07T15:30:18.000Z,"{'urls': [{'start': 281, 'end': 304, 'url': 'h...",1214569958996422656,This is not acceptable.\nWe want their faces a...,en,NaN,190253305,"[{'type': 'quoted', 'id': '1214488973038497792'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
querry1 = gen_request_parameters("(#EndSARS OR #SoroSoke OR #EndPoliceBrutality OR #lekkitollgate OR anti-robbery OR #lekki massacre OR #Endbadgovernance OR #Endswat OR buhari is a bad boy OR sanwa-olu is a murderer OR #LekkiMassacre) lang:en -is:retweet",results_per_call=500, start_time ="2020-01-09", end_time="2020-01-16",
                                tweet_fields="id,author_id,created_at,text,geo,referenced_tweets,in_reply_to_user_id,entities,public_metrics,lang,possibly_sensitive",
                                expansions="author_id,entities.mentions.username,geo.place_id,in_reply_to_user_id,referenced_tweets.id,referenced_tweets.id.author_id",
                                user_fields="id,created_at,location,username,verified,public_metrics",
                                place_fields="country,country_code,full_name,geo,id,name,place_type")


tweetss1 = collect_results(querry1,
                        max_tweets=6000,
                        result_stream_args=search_args)


[print(tweet['text'], end='\n\n') for tweet in tweetss1[0:10]]
tweet_df6= pd.DataFrame(tweetss1)
tweet_df6.head()

When you think you can impress girl with Dubai 😂🤣🤣 #ICameToFunaab #blaqbonez #KhafisGoalGetters #Rashford #nigeria #EndSARS #davido #wizkhalifa #funny https://t.co/9vS43gda2N

Learn from those buffalo whenever SARS raid your neighbourhood illegally. Especially if you can outnumber them.
#EndSARS impunity https://t.co/PNojK51zr0

When robbers are chasing you, but you see SARS ahead so you run back to the armed robbers for safety 🥰
#endsars

Nigerian police (aka SARS) when they see a neatly dressed boy. #EndSARS https://t.co/XIGAsYkqAA

Cool....Kuku start the protest from your house. #EndSARS https://t.co/CNH7txUysV

@abiolaobe @segalink We appreciate you sir @segalink #EndSARS Thanks for always helping niggaz out in times of despair 😩

We don’t want any more innocent souls to die this year.
Let’s #EndSARS and #EndImpunity

Please twitter famz do your thing,I just recently got recruited to supply oil and gas...but I don't have who to supply to...please re-tweet this ...So I can get buyer

,lang,text,entities,public_metrics,id,author_id,possibly_sensitive,created_at,referenced_tweets,in_reply_to_user_id,geo,users,tweets,places,newest_id,oldest_id,result_count,next_token
0,en,When you think you can impress girl with Dubai...,"{'annotations': [{'start': 41, 'end': 45, 'pro...","{'retweet_count': 0, 'reply_count': 0, 'like_c...",1217589634047389702,2681370439,False,2020-01-15T23:29:25.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,en,Learn from those buffalo whenever SARS raid yo...,"{'annotations': [{'start': 34, 'end': 37, 'pro...","{'retweet_count': 1, 'reply_count': 0, 'like_c...",1217564103335608321,1114545372884500482,False,2020-01-15T21:47:58.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,en,"When robbers are chasing you, but you see SARS...","{'annotations': [{'start': 42, 'end': 45, 'pro...","{'retweet_count': 0, 'reply_count': 0, 'like_c...",1217544956329517056,1158161301450952704,False,2020-01-15T20:31:53.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,en,Nigerian police (aka SARS) when they see a nea...,"{'hashtags': [{'start': 63, 'end': 71, 'tag': ...","{'retweet_count': 1, 'reply_count': 0, 'like_c...",1217538179907117058,2983817729,False,2020-01-15T20:04:57.000Z,"[{'type': 'quoted', 'id': '1217529012794183681'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,en,Cool....Kuku start the protest from your house...,"{'hashtags': [{'start': 48, 'end': 56, 'tag': ...","{'retweet_count': 0, 'reply_count': 1, 'like_c...",1217535888504299520,219590858,False,2020-01-15T19:55:51.000Z,"[{'type': 'quoted', 'id': '937378642522451968'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
#import pickle


In [71]:
#filename = 'tweet_result'
#outfile = open(filename,'wb')



In [72]:
#pickle.dump(result,outfile)
#outfile.close()



#### def retrieve_tweets():
    """
    Create a function that gets tweets using the premium search API where matching a certain keyword during a
    certain date range and returns all these tweets as a list
    """

In [ ]:
def get_dt_after(, delta=timedelta(days=7)):
    querry1 = gen_request_parameters("(#EndSARS OR #SoroSoke OR #EndPoliceBrutality OR #lekkitollgate OR anti-robbery OR #lekki massacre OR #Endbadgovernance OR #Endswat OR buhari is a bad boy OR sanwa-olu is a murderer OR #LekkiMassacre) lang:en -is:retweet",results_per_call=500, start_time ="2020-01-09", end_time="2020-01-16",
                                tweet_fields="id,author_id,created_at,text,geo,referenced_tweets,in_reply_to_user_id,entities,public_metrics,lang,possibly_sensitive",
                                expansions="author_id,entities.mentions.username,geo.place_id,in_reply_to_user_id,referenced_tweets.id,referenced_tweets.id.author_id",
                                user_fields="id,created_at,location,username,verified,public_metrics",
                                place_fields="country,country_code,full_name,geo,id,name,place_type")


    tweetss1 = collect_results(querry1,
                        max_tweets=6000,
                        result_stream_args=search_args)


[print(tweet['text'], end='\n\n') for tweet in tweetss1[0:10]]
tweet_df7= pd.DataFrame(tweetss1)
tweet_df7.head()
    

In [84]:
def get_dt_after(start_date, delta=timedelta(days=7)):
    """
    Create a function where you put in a date and it returns a date 7 days after in string format
    """

In [90]:
def create_tweets_df():
    Create a function that Creates a dataframe from the tweets you've retrieved using the retieve_tweets function 
    you created above. This functions should also take a list of datest you want to get tweets from
    """


## Fetch and Clean Data

Use the functions you've created above to collect all the tweets

In [92]:
dates_list = ["2020-12-01",  "2020-12-08"]

In [1]:
tweets_df = create_tweets_df(dates_list, max_tweets=10000)
tweets_df.shape

Make sure to save the tweets after this process, use a file format that won't comprimise the characters in tweets

In [2]:
def delete_duplicates(df):
    """
    Create a function that looks at the tweets in the tweets dataframe and other characters and delete 
    duplicates as sometimes we can get duplicate data/tweets. For example if I tweet something and someone else
    retweets it then this could possibly be a duplicate. Is this useful information? You need to decide of course.
    """

def preprocess_tweet(tweet):
    """
    Create a function that preprocess the text in a single tweet. Suggestions for this function are:
    - correct all multiple white spaces to a single white space
    - convert all urls to string "URL"
    - if the username isn't relevant then convert @username to "AT_USER"
    - converts emoji's to text description of the emoji
    """

In [ ]:
tweets_df = delete_duplicates(tweets_to_analyse)
tweets_df.reset_index(inplace=True, drop=True)
tweets_df['text_clean'] = tweets_df['text'].apply(preprocess_tweet)

## Alt data

In [ ]:
df = pd.read_excel('https://query.data.world/s/rghc6hq5b32ertwgebgwuyx2lcpxvt')

## Tweets Analysis

Now it's time to analyse this data to try and get some interesting insights. Below are some questions you can answer to get started but the best Data Scientists are creative thinkers so think out of th ebox to decide what will be interesting insight

**What locations in Nigeria and around the world were the hashtags popular?**

**How far did the hashtag reach and how deep was this reach?**

**When the did movement start on social media? When did it reach critical mass?**

**What were the most popular words or terms used during the movement?**

**Who were the key influences and proponents involved in the movement?**

**What was the most popular tweet?**

**What is the general sentiment of the tweets about the movement? How has this changed from the start of the movement till now?**

**Were there any bots or bad actors spreading fake news around the movement?**

**What are some examples of this fake news?**

## Network Analysis

Using networkx or any other network graph library, create a social network to identifiy influential people of the movement

## Sentiment Modelling
Train model on labelled twitter sentiment data from another dataset
<br>Next steps:
- Download Sentiment140 data
- preprocess tweet data
- Apply word embeddings to the text
- apply fasttext text classification
- Pickle model
- Apply model to endsars tweets text

### Get data ready for fastText training

Save as .txt file for the fastText classifer

In [ ]:
def prep_data(df, filename=""):
    """
    Create a functions that prepares the endsars data in a format that the fasttext model likes
    """

**Train Model**

**Apply Modelto Test data**

**Score Model**

**Check labels on example tweets**

### Apply trained fastText model to tweets df

In [ ]:
tweets_df['ft_predict'] = tweets_df.text.apply(ft_model.predict)
tweets_df["sentiment"] = tweets_df.ft_predict.apply(lambda x: 1 if x[0][0] == '__label__positive' else
                                                               0)
tweets_df['sentiment_prob'] = tweets_df.ft_predict.apply(lambda x: x[1][0])

In [ ]:
def print_tweet_sentiment(index=0):
    """
    Create function where you either pass the index of the tweet in the tweets_df or a tweet and it returns the 
    sentiment for the tweet
    """